Configuração e leitura

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob 

In [2]:
df = pd.read_csv('data/bronze/dados_brutos.csv')
print(f"Dados originais (Bronze): {df.shape[0]} linhas")

TEXT_COL = 'Texto_Bruto' 
SENTIMENT_COL = 'Sentimento_Bruto'

Dados originais (Bronze): 1600000 linhas


Transformação 1: Limpeza Estrutural e Tipos

In [3]:
# Filtrar colunas relevantes e renomear para nomes limpos
df_clean = df[['Sentimento_Bruto', 'Data_Bruta', 'Texto_Bruto']].copy()
df_clean.columns = ['sentimento_label', 'data_postagem', 'texto_original']

In [4]:
df_clean['data_postagem'] = pd.to_datetime(df_clean['data_postagem'], errors='coerce')
df_clean['sentimento_binario'] = df_clean['sentimento_label'].apply(lambda x: 1 if x == 4 else 0)

# C. Remoção de Duplicatas e Nulos
linhas_antes = len(df_clean)
df_clean.drop_duplicates(inplace=True)
df_clean.dropna(inplace=True) 
linhas_depois = len(df_clean)

print(f"\nLinhas após limpeza estrutural: {linhas_depois}. Duplicatas/Nulos removidos: {linhas_antes - linhas_depois}")


Linhas após limpeza estrutural: 1600000. Duplicatas/Nulos removidos: 0


Transformação 2: Processamento de Linguagem Natural

In [5]:
def clean_text(text):
    text = str(text)
    
    text = re.sub(r'http\S+|www.\S+', '', text)
    
    text = re.sub(r'@\w+|#\w+', '', text)
    
    text = re.sub(r'[^\w\s]', '', text).lower()
    
    return text.strip()

df_clean['texto_limpo'] = df_clean['texto_original'].apply(clean_text)

print("\nTexto Original vs. Texto Limpo:")
print(df_clean[['texto_original', 'texto_limpo']].head())


Texto Original vs. Texto Limpo:
                 texto_original                 texto_limpo
0  Mon Apr 06 22:19:45 PDT 2009  mon apr 06 221945 pdt 2009
1  Mon Apr 06 22:19:49 PDT 2009  mon apr 06 221949 pdt 2009
2  Mon Apr 06 22:19:53 PDT 2009  mon apr 06 221953 pdt 2009
3  Mon Apr 06 22:19:57 PDT 2009  mon apr 06 221957 pdt 2009
4  Mon Apr 06 22:19:57 PDT 2009  mon apr 06 221957 pdt 2009


Transformação 3: Análise de Sentimento

In [6]:
# O score vai de -1 (muito negativo) a +1 (muito positivo)
def get_textblob_sentiment(text):
    return TextBlob(text).sentiment.polarity

# Criar a coluna métrica principal (Polaridade)
df_clean['score_sentimento'] = df_clean['texto_limpo'].apply(get_textblob_sentiment)

print("\nMétrica de Sentimento Gerada:")
print(df_clean[['texto_limpo', 'score_sentimento']].head())


Métrica de Sentimento Gerada:
                  texto_limpo  score_sentimento
0  mon apr 06 221945 pdt 2009               0.0
1  mon apr 06 221949 pdt 2009               0.0
2  mon apr 06 221953 pdt 2009               0.0
3  mon apr 06 221957 pdt 2009               0.0
4  mon apr 06 221957 pdt 2009               0.0


Finalização e Carga na Silver Layer

In [7]:
# Adicionar metadado de processamento
df_clean['data_processamento'] = datetime.now()

# Salvar na camada Silver
output_path = 'data/silver/dados_limpos.csv'
df_clean.to_csv(output_path, index=False)

print(f"\nDados Silver salvos com sucesso: {df_clean.shape[0]} linhas")


Dados Silver salvos com sucesso: 1600000 linhas
